<a href="https://colab.research.google.com/github/arjundased/hugo-northendlab-forestry/blob/master/trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install alpha_vantage
!pip install fbprophet


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached fbprophet-0.7.1.tar.gz (64 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for fbprophet
  Running setup.py clean for fbprophet
Failed to build fbprophet
  error: subprocess-exited-with-error
  
  × Running setup.py install for fbprophet did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for fbprophet ... error
error: legacy-install-failure

× Encountered error while trying to install 

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
from alpha_vantage.timeseries import TimeSeries

plt.style.use('fivethirtyeight')


def get_stock_data(stock_symbol, start_date, end_date, api_key):
    try:
        ts = TimeSeries(key=api_key, output_format='pandas')
        data, _ = ts.get_daily_adjusted(symbol=stock_symbol, outputsize='full')
        data = data[start_date:end_date]
        data = data.rename(columns={'4. close': 'Close'})
        return data
    except ValueError as e:
        print("Error fetching data:", e)
        return None


def calculate_bollinger_bands(data, window=20, std=2):
    rolling_mean = data['Close'].rolling(window=window).mean()
    rolling_std = data['Close'].rolling(window=window).std()
    data['Upper_Band'] = rolling_mean + (rolling_std * std)
    data['Lower_Band'] = rolling_mean - (rolling_std * std)
    return data


def generate_buy_sell_signals(data):
    buy_signal = (data['Close'] < data['Lower_Band'])
    sell_signal = (data['Close'] > data['Upper_Band'])
    data['Signal'] = np.where(buy_signal, 1, np.where(sell_signal, -1, 0))
    data['Buy'] = np.where(data['Signal'] == 1, data['Close'], np.nan)
    data['Sell'] = np.where(data['Signal'] == -1, data['Close'], np.nan)
    return data


def plot_stock_data(data, stock_symbol, start_date, end_date):
    fig, ax1 = plt.subplots(figsize=(14, 8))
    fig.suptitle(stock_symbol, fontsize=10, backgroundcolor='blue', color='white')
    ax1.plot(data['Close'], label='Close Price', linewidth=0.5, color='blue')
    ax1.scatter(data.index, data['Buy'], label='Buy', marker='^', color='green', alpha=1)
    ax1.scatter(data.index, data['Sell'], label='Sell', marker='v', color='red', alpha=1)
    ax1.plot(data['Upper_Band'], label='Upper Band', linewidth=0.5, color='black')
    ax1.plot(data['Lower_Band'], label='Lower Band', linewidth=0.5, color='black')
    ax1.legend()
    ax1.grid()
    ax1.set_xlabel(f'{start_date} - {end_date}', fontsize=18)
    ax1.set_ylabel('Close Price USD ($)', fontsize=18)
    plt.tight_layout()
    plt.show()

def calculate_strategy_metrics(data):
    trades = data[data['Signal'] != 0]
    total_trades = len(trades)
    winning_trades = len(trades[trades['Signal'] == 1])
    losing_trades = len(trades[trades['Signal'] == -1])
    win_rate = winning_trades / total_trades * 100

    data['Trade_PnL'] = data['Close'].shift(-1) - data['Close']
    data['Trade_PnL'] = data['Trade_PnL'] * data['Signal']
    average_gain = data[data['Trade_PnL'] > 0]['Trade_PnL'].mean()
    average_loss = data[data['Trade_PnL'] < 0]['Trade_PnL'].mean()
    profit_factor = abs(average_gain / average_loss)

    total_return = data['Trade_PnL'].sum()
    initial_investment = data.iloc[0]['Close']
    final_investment = data.iloc[-1]['Close']
    buy_and_hold_return = final_investment - initial_investment

    return {
        'Total Trades': total_trades,
        'Winning Trades': winning_trades,
        'Losing Trades': losing_trades,
        'Win Rate': win_rate,
        'Average Gain': average_gain,
        'Average Loss': average_loss,
        'Profit Factor': profit_factor,
        'Total Return': total_return,
        'Buy and Hold Return': buy_and_hold_return,
    }


if __name__ == "__main__":
    stock_symbol = 'AAPL'
    start_date = date(2017, 8, 4)
    end_date = date.today()
    api_key = 'FCBONE1F4VHPXS1W'  # Replace with your Alpha Vantage API key

    stock_data = get_stock_data(stock_symbol, start_date, end_date, api_key)

    if stock_data is not None:
        stock_data = calculate_bollinger_bands(stock_data)
        stock_data = generate_buy_sell_signals(stock_data)
        strategy_metrics = calculate_strategy_metrics(stock_data)

        print("\nStrategy Metrics:")
        for key, value in strategy_metrics.items():
            print(f"{key}: {value}")

        plot_stock_data(stock_data, stock_symbol, start_date, end_date)


ZeroDivisionError: ignored